In [1]:
file_path = r"C:\Users\james\Documents\GitHub\Traffic_Volume_Prediction"

In [2]:
import pandas as pd

train = pd.read_csv(file_path + r"\Data\train.csv")
test = pd.read_csv(file_path + r"\Data\test.csv")

In [3]:
# X_train과 y_train 설정
X_train = train.drop(columns=['1005004000_velocity'])
y_train = train[['1005004000_velocity']]

# X_test와 y_test 설정
X_test = test.drop(columns=['1005004000_velocity'])
y_test = test[['1005004000_velocity']]

In [4]:
# 타겟 컬럼과 시간 시차 설정
TARGET = '1005004000_velocity'
HORIZON = 24  # 24시간 시차

# 24시간 시차 적용하여 데이터 생성
train['y_shifted'] = train[TARGET].shift(-HORIZON)  # 24시간 이후 값을 타겟으로 설정
test['y_shifted'] = test[TARGET].shift(-HORIZON)

# 비어 있는 데이터 제거
train = train.dropna(subset=['y_shifted'])
test = test.dropna(subset=['y_shifted'])

# X_train, y_train 설정
X_train = train.drop(columns=[TARGET, 'y_shifted'])
y_train = train[['y_shifted']]

# X_test, y_test 설정
X_test = test.drop(columns=[TARGET, 'y_shifted'])
y_test = test[['y_shifted']]

# 결과 출력 (확인용)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (19261, 21)
y_train shape: (19261, 1)
X_test shape: (696, 21)
y_test shape: (696, 1)


In [5]:
# 넷 다 dataframe
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(19261, 21) (19261, 1) (696, 21) (696, 1)


In [6]:
# 처리 방식 정의
fill_methods = {
    '1050003300_velocity': 'ffill',
    '1070000200_velocity': 'ffill',
    '1070000200_velocity': 'ffill',
    '1070000500_velocity': 'ffill',
    '1070001600_velocity': 'ffill',
    'F-03 유입_traffic': 'ffill',
    'F-04 유출_traffic': 'ffill',
    '강수량(mm)' : 0,
    '풍속(m/s)' : 'average',
    '적설(cm)': 0
}

for df in [X_train, X_test]:
    # for loop로 결측값 처리
    for column, method in fill_methods.items():
        if method == 'average':  # 평균으로 채우기
            df[column] = df[column].fillna(df[column].mean())
        elif method == 'ffill':  # 이전 값으로 채우기
            df[column] = df[column].ffill()
        else:  # 특정 값으로 채우기
            df[column] = df[column].fillna(method)

In [7]:
# 모든 열을 float32로 변환
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19261 entries, 0 to 19260
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   1050003300_velocity  19261 non-null  float32
 1   1050020400_velocity  19261 non-null  float32
 2   1070000200_velocity  19261 non-null  float32
 3   1070000500_velocity  19261 non-null  float32
 4   1070001600_velocity  19261 non-null  float32
 5   F-03 유입_traffic      19261 non-null  float32
 6   F-04 유출_traffic      19261 non-null  float32
 7   기온(°C)               19261 non-null  float32
 8   강수량(mm)              19261 non-null  float32
 9   풍속(m/s)              19261 non-null  float32
 10  습도(%)                19261 non-null  float32
 11  적설(cm)               19261 non-null  float32
 12  Year                 19261 non-null  float32
 13  is_weekend           19261 non-null  float32
 14  is_holiday           19261 non-null  float32
 15  is_morning_rush      19261 non-null  floa

In [9]:
X_train.isnull().sum().sum()

0

In [15]:
import torch
import torch.nn as nn
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

# 설정 값
N_TEST = 12
N_SPLIT = 27
INPUT_SIZE = 21  # X_train의 feature 수
PATCH_SIZE = 16  # Patch 크기
HIDDEN_SIZE = 128
OUTPUT_SIZE = 1  # y_train의 feature 수
NUM_HEADS = 4
NUM_LAYERS = 3
EPOCHS = 50
LEARNING_RATE = 0.001

# CUDA 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# TimeSeriesSplit 설정
tscv = TimeSeriesSplit(n_splits=N_SPLIT)

# MAPE 점수 리스트
mape_scores = []

# PatchTST 모델 정의
class PatchTST(nn.Module):
    def __init__(self, input_size, patch_size, hidden_size, num_heads, num_layers, output_size):
        super(PatchTST, self).__init__()
        self.patch_size = patch_size
        self.input_proj = nn.Linear(patch_size, hidden_size)

        # Transformer Encoder 설정
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_size, 
            nhead=num_heads, 
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # 1. 패치 분할
        batch_size, seq_len, feature_size = x.shape
        x = x.unfold(dimension=1, size=self.patch_size, step=self.patch_size)
        x = x.permute(0, 2, 1, 3).reshape(batch_size, -1, self.patch_size)
        
        # 2. 패치 임베딩
        x = self.input_proj(x)

        # 3. Transformer Encoder
        x = self.transformer_encoder(x)

        # 4. 최종 출력
        out = self.fc(x.mean(dim=1))  # [Batch, Hidden] → [Batch, Output]
        return out

# 슬라이딩 윈도우 생성 함수
def create_sliding_windows(data, window_size):
    n_samples, n_features = data.shape
    windows = []
    for i in range(n_samples - window_size + 1):
        window = data[i:i + window_size]
        windows.append(window)
    return torch.tensor(np.array(windows), dtype=torch.float32)

# PatchTST 모델 초기화 및 CUDA로 이동
model = PatchTST(INPUT_SIZE, PATCH_SIZE, HIDDEN_SIZE, NUM_HEADS, NUM_LAYERS, OUTPUT_SIZE).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# TimeSeriesSplit을 이용한 검증
for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train)):
    if fold < N_SPLIT - N_TEST:
        continue

    # Train/Validation 데이터 분리
    X_tr, X_val = X_train.iloc[train_idx].values, X_train.iloc[val_idx].values
    y_tr, y_val = y_train.iloc[train_idx].values, y_train.iloc[val_idx].values

    # 슬라이딩 윈도우 변환 (PatchTST 입력 형식)
    X_tr_windows = create_sliding_windows(X_tr, PATCH_SIZE).to(device)  # [Batch, Patch, Features]
    X_val_windows = create_sliding_windows(X_val, PATCH_SIZE).to(device)  # [Batch, Patch, Features]
    y_tr_tensor = torch.tensor(y_tr[PATCH_SIZE - 1:], dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val[PATCH_SIZE - 1:], dtype=torch.float32).to(device)

    # PatchTST 학습
    for epoch in range(EPOCHS):
        model.train()
        optimizer.zero_grad()
        y_pred = model(X_tr_windows)
        loss = criterion(y_pred, y_tr_tensor)
        loss.backward()
        optimizer.step()

    # 검증
    model.eval()
    with torch.no_grad():
        y_val_pred = model(X_val_windows).cpu().squeeze().numpy()
        mape = mean_absolute_percentage_error(y_val[PATCH_SIZE - 1:], y_val_pred)
        mape_scores.append(mape)
        print(f"Fold {fold + 1}, Validation MAPE: {mape:.4f}")

# 테스트 세트에 대한 최종 예측
X_test_windows = create_sliding_windows(X_test.values, PATCH_SIZE).to(device)
y_test_tensor = torch.tensor(y_test.values[PATCH_SIZE - 1:], dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    y_test_pred = model(X_test_windows).cpu().squeeze().numpy()
    test_mape = mean_absolute_percentage_error(y_test.values[PATCH_SIZE - 1:], y_test_pred)

# 결과 출력
print("Results: ")
print(f"Average Validation MAPE: {np.mean(mape_scores):.4f}")
print(f"Validation MAPE Std Dev: {np.std(mape_scores):.4f}")
print(f"Final Test MAPE: {test_mape:.4f}")

# GPU 메모리 비우기
torch.cuda.empty_cache()
print("CUDA memory cleared.")

Using device: cuda


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.76 GiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 8.77 GiB is allocated by PyTorch, and 1.51 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)